In [1]:
import os
from itertools import islice

import numpy as np
import pandas as pd

from skimage.external.tifffile import imread

import ipywidgets as widgets

In [2]:
bigDir = r'S:\CookLab\Liu\20190816_organizedData_MCM_loading\DATA'
cellDataFile=r'S:\CookLab\Liu\20190816_organizedData_MCM_loading\sel_cells_v1.csv'

In [3]:
cellData=pd.read_csv(cellDataFile)
cellData.head()

,Unnamed: 0,file,original cell name,Amy_mod,movie,x,y,ab1 (heterochromatin),ab1 ch,ab2 (MCM),...,DHB_cyto,DHB_ratio,DHB_len,physical_age_cleaned,physicalAgeMatch,time_mismatch,track_vs_measure,molecularAgeMatch,molecularAge_mismatch,molecularAge_ratio
0,0,20190625_cell-00.czi,7LE1(B),1130/10,190619-cdc6-dhb-pcna002xy09,172.0,956.0,HP1B,1.0,MCM3,...,"[638.2180000000001, 716.823, 810.238, 684.439,...","[0.3245789670420766, 0.3256659514179053, 0.564...",5.0,30.0,1.0,-2.0,0.0,True,0.0,0.82
1,1,20190625_cell-01.czi,7L-E1(T),0,190619-cdc6-dhb-pcna002xy09,158.0,906.0,HP1B,1.0,MCM3,...,"[638.2180000000001, 639.193, 962.079, 759.0060...","[0.3245789670420766, 0.2901918001758785, 0.738...",5.0,30.0,1.0,-2.0,0.0,True,0.0,0.82
2,2,20190625_cell-02.czi,7L-E2(B),0,190619-cdc6-dhb-pcna002xy09,832.0,252.0,HP1B,1.0,MCM3,...,"[580.302, 662.867, 973.7439999999999, 801.477,...","[0.31538769236789416, 0.4297265794161806, 0.97...",13.0,110.0,1.0,-2.0,0.0,True,0.0,0.43
3,3,20190625_cell-03.czi,7L-E2(T),0,190619-cdc6-dhb-pcna002xy09,888.0,160.0,HP1B,1.0,MCM3,...,"[548.491, 508.965, 907.77, 722.7660000000001, ...","[0.30119987699201545, 0.38937536530358063, 0.9...",13.0,110.0,1.0,-2.0,0.0,True,0.0,0.50
4,5,20190625_cell-05.czi,7L-G2,0,190619-cdc6-dhb-pcna002xy09,172.0,324.0,HP1B,1.0,MCM3,...,"[707.5169999999999, 757.383, 972.8560000000001...","[0.2633849381683813, 0.3234133643516514, 0.817...",118.0,1160.0,1.0,-2.0,0.0,True,0.0,1.48


In [6]:
i=37

# Progress Bar
progBar=widgets.IntProgress(
    value=i,
    min=0,
    max=len(cellData),
    step=1,
    description='Progress:',
    orientation='horizontal'
)
display(progBar)

for i,myCell in islice(cellData.iterrows(),i,None):
    
    # create results folder if needed
    newResFolder = os.path.dirname(myCell.path).replace('data_tiff','results')

    try:
        os.mkdir(newResFolder)
    except:
        pass
    
    # create df to store results
    df = pd.DataFrame(columns = ['nucleus_Otsu_volume','nucleus_Otsu_ch1','nucleus_Otsu_ch2','nucleus_Otsu_ch3',
                                 'nucleus_Li_volume','nucleus_Li_ch1','nucleus_Li_ch2','nucleus_Li_ch3',
                                 'hetChrom_gestalt_volume','hetChrom_gestalt_ch1','hetChrom_gestalt_ch2','hetChrom_gestalt_ch3',
                                 'hetChrom_manual_volume','hetChrom_manual_ch1','hetChrom_manual_ch2','hetChrom_manual_ch3',
                                 'Slice'])
    
    # open nucleus mask Otsu
    tempPath=myCell.path
    tempPath=tempPath.replace('data_tiff','segmentation_nucleus_Otsu')
    tempPath=tempPath.replace('.tif','_nucleus.tif')
    nucleusMaskOtsu=imread(tempPath)
    
    # make nucleus mask binary
    nucleusMaskOtsu = nucleusMaskOtsu.astype(bool)
    
    # open nucleus mask Otsu
    tempPath=myCell.path
    tempPath=tempPath.replace('data_tiff','segmentation_nucleus_Li')
    tempPath=tempPath.replace('.tif','_nucleus.tif')
    nucleusMaskLi=imread(tempPath)

    # make nucleus mask binary
    nucleusMaskLi = nucleusMaskLi.astype(bool)
    
    # open hetChrom gestalt mask
    tempPath=myCell.path
    tempPath=tempPath.replace('data_tiff','segmentation_gestalt')
    tempPath=tempPath.replace('.tif','_hetChrom.tif')
    hetChromGestalt=imread(tempPath)

    # make nucleus mask binary
    hetChromGestalt = hetChromGestalt.astype(bool)
    # clean with Otsu mask
    hetChromGestalt[~nucleusMaskOtsu]=0
    
    # open hetChrom manual mask
    tempPath=myCell.path
    tempPath=tempPath.replace('data_tiff','segmentation_manual')
    tempPath=tempPath.replace('.tif','_hetChrom.tif')
    hetChromManual=imread(tempPath)

    # make nucleus mask binary
    hetChromManual = hetChromManual.astype(bool)
    # clean with Otsu mask
    hetChromManual[~nucleusMaskOtsu]=0
    
    # open tiff image of a cell
    myImage=imread(myCell.path)    
    ch1 = myImage[:, 0, :, :] #1st channel - hetChrom (as stained by HP1B or H3K9Me3)
    ch2 = myImage[:, 1, :, :] #2nd channel - MCM
    ch3 = myImage[:, 2, :, :] #3rd channel - DAPI
    
    # calculate signals
    masks=['nucleusMaskOtsu','nucleusMaskLi','hetChromGestalt','hetChromManual']
    masksNames=['nucleus_Otsu','nucleus_Li','hetChrom_gestalt','hetChrom_manual']
    signals=[None,'ch1','ch2','ch3']
    signalsNames=['volume','ch1','ch2','ch3']
    
    df.Slice = range(ch1.shape[0])
    
    for myMask,myMaskName in zip(masks,masksNames):
        
        for mySignal,mySignalName in zip(signals,signalsNames):
            
            if mySignal is None:
                mySignal=myMask
            
            myString=f'{mySignal}[~{myMask}]=0'
            exec(myString)
            myString=f'df.{myMaskName}_{mySignalName}={mySignal}.sum(-1).sum(-1)'
            exec(myString)

    
    # save signals (per frame)
    dfSavePath=myCell.path
    dfSavePath=dfSavePath.replace('data_tiff','results')
    dfSavePath=dfSavePath.replace('.tif',f'_vol_gest_man.csv')
    df.to_csv(dfSavePath)

    # add info to cell info file
    for myMeasurement in df.columns:
        
        myString=f"cellData.loc[i,'{myMeasurement}']=np.sum(df.{myMeasurement})"
        exec(myString)
    
    progBar.value=progBar.value+1

IntProgress(value=37, description='Progress:', max=300)

In [7]:
i

299

In [8]:
cellData.to_csv(cellDataFile.replace('v1','v2'))